### Data Generator

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [2]:
train_idg = image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True
)

val_idg = image.ImageDataGenerator(
    rescale=1./255
)

In [3]:
train_generator = train_idg.flow_from_directory(
    '../Dataset/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 273 images belonging to 2 classes.


In [4]:
val_generator = val_idg.flow_from_directory(
    '../Dataset/test',
    target_size=(224,224),
    batch_size=24,
    class_mode='binary'
)

Found 68 images belonging to 2 classes.


### Scratch Model

In [5]:
from keras.layers import *
from keras.models import Sequential

In [6]:
model = Sequential()

In [7]:
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [9]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

- Train Model

In [10]:
from keras.callbacks import ModelCheckpoint
import os

In [11]:
if not os.path.exists('../Weights/Scratch'):
    os.mkdir('../Weights/Scratch')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [12]:
checkpoint = ModelCheckpoint('../Weights/Scratch/model.{epoch}_{val_accuracy:.2f}.h5', 
                             monitor='val_accuracy', 
                             verbose=1, 
                             mode='auto',
                             save_weights_only=True
                            )

In [13]:
hist = model.fit(
    train_generator,
    steps_per_epoch=9,
    epochs=10,
    validation_data=val_generator,
    validation_steps=2,
    callbacks=checkpoint
)

Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 2.5075 - accuracy: 0.4652
Epoch 00001: saving model to ../Weights/Scratch/model.1_0.56.h5
9/9 [==============================] - 87s 10s/step - loss: 2.5075 - accuracy: 0.4652 - val_loss: 0.6920 - val_accuracy: 0.5625
Epoch 2/10
9/9 [==============================] - ETA: 0s - loss: 0.6899 - accuracy: 0.5897
Epoch 00002: saving model to ../Weights/Scratch/model.2_0.58.h5
9/9 [==============================] - 86s 10s/step - loss: 0.6899 - accuracy: 0.5897 - val_loss: 0.6896 - val_accuracy: 0.5833
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 0.6896 - accuracy: 0.5641
Epoch 00003: saving model to ../Weights/Scratch/model.3_0.60.h5
9/9 [==============================] - 85s 9s/step - loss: 0.6896 - accuracy: 0.5641 - val_loss: 0.6911 - val_accuracy: 0.6042
Epoch 4/10
9/9 [==============================] - ETA: 0s - loss: 0.6676 - accuracy: 0.6154
Epoch 00004: saving model to ../Weights/Scratch/model.4_0

In [14]:
x,y=val_generator.next()

In [15]:
model.evaluate(x,y)

1/1 [==============================] - 0s 10ms/step - loss: 0.2321 - accuracy: 0.9583


[0.2320740669965744, 0.9583333134651184]

- Save Model History

In [16]:
import numpy as np
import os

In [17]:
if not os.path.exists('../Model_history'):
    os.mkdir('../Model_history')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [ ]:
np.save('../Model_history/scratch_file.npy',hist.history)
read_dictionary = np.load('../Model_history/scratch_file.npy',allow_pickle='TRUE').item()
print(read_dictionary.keys())